In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt 
import seaborn as sn
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import GridSearchCV
import pydotplus as pdot
from IPython.display import Image

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pydotplus

In [ ]:
diabetes = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")
diabetes.info()

print(diabetes.shape)

print(diabetes.head(5))

## Exploratory Data Analysis: EDA

### EDA#1: Correlation Matrix

In [ ]:
# Create Correlation Matrix
corrMatrix = diabetes.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

<b> The correlation matrix suggests that there is notable positive corelation of value 0.47 between Glucose and Outcome. <b>

## Split predictors and target variable

In [ ]:
# Split predictors and target variable

X = diabetes.loc[:, diabetes.columns != 'Outcome']
y = diabetes["Outcome"]

# Split data into Test & Training dataset
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.20)

### Build a Confusion Matrix

In [ ]:
##Importing the metrics  from sklearn import metrics 
##Defining the matrix to draw the confusion matrix from actual and  ##predicted class labels 
def draw_cm( actual, predicted):
    #Invoking confusion_matrix from metric package. The matrix  
    #will be oriented as [1,0] i.e. the classes with label 1 will be  
    #represented by the first row and 0 as second row  
    cm = metrics.confusion_matrix( actual, predicted, [1,0])  
    #Confusion will be plotted as heatmap for better visualization  
    #The labels are configured to better interpretation from the plot  
    sn.heatmap(cm, annot=True, fmt='.2f',  
               xticklabels = ["Diabetes-Yes", "Diabetes-No"],  
               yticklabels = ["Diabetes-Yes", "Diabetes-No"])  
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

### Build / Train Random Forest Model

In [ ]:
## Initializing the Random Forest Classifier with max_depth and n_estimators
rf_classifier = RandomForestClassifier(max_depth=10, n_estimators=10)
rf_classifier.fit(train_X, train_y)

### Calculate Precision, Recall, f1-score, Support values

In [ ]:
pred_y = rf_classifier.predict(test_X)
print( metrics.classification_report( test_y, pred_y)) 

### Print confusion matrix

In [ ]:
#Print confusion matrix
cm = draw_cm(test_y, pred_y)

### Finding important Features

In [ ]:
#Create a dataframe to store the features and their corresponding #importances 
feature_rank = pd.DataFrame( {'feature': train_X.columns,  'importance': rf_classifier.feature_importances_  }) 
##Sorting the features based on their importances with most  ##important feature at top. 
feature_rank = feature_rank.sort_values('importance', ascending =  False) 
plt.figure(figsize=(8, 6)) 
#plot the values 
sn.barplot( y = 'feature', x = 'importance', data = feature_rank[0:10]);

## Build / Train Decision Tree Classifier

In [ ]:
## Building Decision Tree Classifier using Gini Criteria
clf_tree = DecisionTreeClassifier(criterion = 'gini', max_depth = 4)
clf_tree.fit(train_X, train_y)

### Calculate Precision, Recall, f1-score, Support values

In [ ]:
pred_y = clf_tree.predict(test_X)
print( metrics.classification_report( test_y, pred_y)) 

In [ ]:
## Displaying the tree
export_graphviz(clf_tree,
               out_file = "chd_tree.odt",
               feature_names = train_X.columns,
               filled = True)

## Create the image file
chd_tree_graph = pdot.graphviz.graph_from_dot_file('chd_tree.odt')
chd_tree_graph.write_jpg('chd_tree.png')
## Render the png file
Image(filename='chd_tree.png')

In [ ]:
#Print confusion matrix
cm = draw_cm(test_y, pred_y)